<a href="https://colab.research.google.com/github/Luiz-sj/Projeto-Final---Bootcamp-Alura-Data-Science-2021/blob/main/Cap%C3%ADtulo_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Seção 3.1:**

#**Otimização de hiperparâmetros**

# A função `sklearn.model_selection.GridSearchCV()`, é baseada no algoritimo de validação cruzada, mas tem como objetivo realizar diversas modelagens levando em consideração a variação de diversos hiperparâmetros. Entre suas diversas saídas, é possível obter a configuração de hiperparâmetros otimizada. Os códigos a seguir ilustram a execução do algoritimo para os modelos estudados nesse trabalho.

#**Aviso**: algumas células de execução deste notebook podem levar um tempo maior para execução do que as células de capítulos anteriores.

In [16]:
#Importando bibliotecas básicas, a função de validação cruzada e o data frame de trabalho
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

#Validação cruzada
from sklearn.model_selection import cross_validate
def run_model_cv(model, df, n_splits, n_repeats):

  X = df.drop(['PATIENT_VISIT_IDENTIFIER', 'WINDOW', 'ICU', 'ICU_norm'], axis=1)
  y = df['ICU_norm']

  metrics = cross_validate(model, X, y, 
                           cv=RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats), 
                           scoring='roc_auc',
                           return_train_score=True)

  train_auc_mean = np.mean(metrics['train_score'])
  test_auc_mean = np.mean(metrics['test_score'])

  print(f"Mean train AUC: {train_auc_mean}. Mean test AUC: {test_auc_mean}")

  return train_auc_mean, test_auc_mean

#data frame
df = pd.read_csv('https://github.com/Luiz-sj/Projeto-Final---Bootcamp-Alura-Data-Science-2021/raw/main/Dados/df_treated.csv').drop(['Unnamed: 0'], axis=1)

df

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_DIFF,BLAST_MEDIAN,BLAST_DIFF,CALCIUM_MEDIAN,CALCIUM_DIFF,CREATININ_MEDIAN,CREATININ_DIFF,FFA_MEDIAN,FFA_DIFF,GGT_MEDIAN,GGT_DIFF,GLUCOSE_MEDIAN,GLUCOSE_DIFF,HEMATOCRITE_MEDIAN,HEMATOCRITE_DIFF,HEMOGLOBIN_DIFF,...,SAT02_ARTERIAL_DIFF,SAT02_VENOUS_MEDIAN,SAT02_VENOUS_DIFF,SODIUM_MEDIAN,SODIUM_DIFF,TGO_MEDIAN,TGO_DIFF,TGP_DIFF,TTPA_MEDIAN,TTPA_DIFF,UREA_MEDIAN,UREA_DIFF,DIMER_MEDIAN,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,WINDOW,ICU,ICU_norm
0,0,1,50,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,-1.0,-1.0,-1.0,-1.000000,-1.0,-0.317073,-1.0,-0.317073,-1.0,-0.938950,-1.0,-1.0,-1.0,0.183673,-1.0,-0.868365,-1.0,-0.742004,-1.0,-0.945093,-1.0,-0.891993,-1.0,0.090147,-1.0,-1.0,...,-1.0,0.345679,-1.0,-0.028571,-1.0,-0.997201,-1.0,-1.0,-0.825613,-1.0,-0.836145,-1.0,-0.994912,-1.0,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0,1
1,2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,-1.0,-1.0,-1.0,-1.000000,-1.0,-0.317073,-1.0,-0.317073,-1.0,-0.938950,-1.0,-1.0,-1.0,0.357143,-1.0,-0.912243,-1.0,-0.742004,-1.0,-0.958528,-1.0,-0.780261,-1.0,0.144654,-1.0,-1.0,...,-1.0,0.345679,-1.0,0.085714,-1.0,-0.995428,-1.0,-1.0,-0.846633,-1.0,-0.836145,-1.0,-0.978029,-1.0,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,-0.525773,-0.5125,-0.111111,-0.714286,0.604396,0.959596,-0.435897,-0.491892,0.000000,-0.575758,0.101449,1.000000,-0.547826,-0.533742,-0.603053,-0.764706,-1.000000,0-2,0,1
2,3,0,30,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.263158,-1.0,-1.0,-1.0,-1.000000,-1.0,-0.317073,-1.0,-0.317073,-1.0,-0.972789,-1.0,-1.0,-1.0,0.326531,-1.0,-0.968861,-1.0,-0.194030,-1.0,-0.316589,-1.0,-0.891993,-1.0,-0.203354,-1.0,-1.0,...,-1.0,0.580247,-1.0,0.200000,-1.0,-0.989549,-1.0,-1.0,-0.846633,-1.0,-0.937349,-1.0,-0.978029,-1.0,0.012346,-0.369231,-0.528302,-0.457627,-0.285714,0.684211,0.175258,-0.1125,-0.384615,-0.357143,0.208791,0.878788,-0.299145,-0.556757,-0.626866,-0.515152,-0.420290,0.684211,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0,0
3,4,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,-1.0,-1.0,-1.0,-1.000000,-1.0,-0.317073,-1.0,-0.317073,-1.0,-0.935113,-1.0,-1.0,-1.0,0.357143,-1.0,-0.913659,-1.0,-0.829424,-1.0,-0.938084,-1.0,-0.851024,-1.0,0.358491,-1.0,-1.0,...,-1.0,0.345679,-1.0,0.142857,-1.0,-0.998507,-1.0,-1.0,-0.846633,-1.0,-0.903614,-1.0,-1.000000,-1.0,0.333333,-0.153846,0.160377,-0.593220,0.285714,0.868421,0.443299,0.0000,0.196581,-0.571429,0.538462,0.939394,-0.076923,-0.351351,-0.044776,-0.575758,0.072464,0.894737,-1.000000,-0.877301,-0.923664,-0.882353,-0.952381,0-2,0,0
4,5,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,-1.0,-1.0,-1.0,-1.000000,-1.0,-0.317073,-1.0,-0.317073,-1.0,-0.938950,-1.0,-1.0,-1.0,0.357143,-1.0,-0.891012,-1.0,-0.742004,-1.0,-0.958528,-1.0,-0.891993,-1.0,0.291405,-1.0,-1.0,...,-1.0,0.345679,-1.0,0.085714,-1.0,-0.997947,-1.0,-1.0,-0.846633,-1.0,-0.884337,-1.0,-1.000000,-1.0,-0.037037,-0.538462,-0.537736,-0.525424,-0.196429,0.815789,0.030928,-0.3750,-0.401709,-0.4

In [2]:
#Definindo a função de otimização
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

def optimization (model, param_grid, df, n_splits, n_repeats):

  X = df.drop(['PATIENT_VISIT_IDENTIFIER', 'WINDOW', 'ICU', 'ICU_norm'], axis=1)
  y = df['ICU_norm']

  search = GridSearchCV(model,
                        param_grid = param_grid,
                        cv = RepeatedStratifiedKFold(n_splits = n_splits, n_repeats=n_repeats), 
                        scoring='roc_auc',
                        return_train_score=True)
  search.fit(X, y)

  return search.best_params_  



In [3]:
#Executando a otimização para a regressão logística
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(max_iter=1000)

param_grid = {
  "penalty" : ['l1', 'l2', 'elasticnet', 'none'],
  "solver" : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
  "multi_class" : ['auto', 'ovr', 'multinomial']
}

optimized = optimization(model_lr, param_grid, df, 5, 10)

optimized

{'multi_class': 'auto', 'penalty': 'l2', 'solver': 'lbfgs'}

In [4]:
#Inserindo os valores otimizados no modelo
model_lr_optimized = LogisticRegression(max_iter=1000,
                                        penalty = optimized['penalty'],
                                        solver = optimized['solver'],
                                        multi_class = optimized['multi_class'])

run_model_cv(model_lr_optimized, df, 5, 10)

Mean train AUC: 0.8537604957994493. Mean test AUC: 0.7715405432346221


(0.8537604957994493, 0.7715405432346221)

In [5]:
#Realizando o processo para o modelo de árvore de decisão
from sklearn.tree import DecisionTreeClassifier

model_tree = DecisionTreeClassifier()

param_grid = {
  "max_depth" : [4, 8, 16, 32],
  "min_samples_split" : [32, 64, 128, 256],
  "min_samples_leaf" : [32, 64, 128, 256],
  "splitter" : ["best", "random"],
  "criterion" : ["gini", "entropy"]
}

optimized = optimization(model_tree, param_grid, df, 5, 10)

optimized

{'criterion': 'gini',
 'max_depth': 8,
 'min_samples_leaf': 32,
 'min_samples_split': 64,
 'splitter': 'best'}

In [6]:
#Inserindo os valores otimizados no modelo
model_tree_optimized = DecisionTreeClassifier(criterion = optimized['criterion'],
                                              max_depth = optimized['max_depth'],
                                              min_samples_leaf = optimized['min_samples_leaf'],
                                              min_samples_split = optimized['min_samples_split'],
                                              splitter = optimized['splitter'])

run_model_cv(model_lr_optimized, df, 5, 10)

Mean train AUC: 0.8535751244855868. Mean test AUC: 0.7703316279796543


(0.8535751244855868, 0.7703316279796543)

In [7]:
#Realizando o processo para o modelo de random forest
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()

param_grid = {
  "n_estimators" : [8, 16, 32, 64],
  "max_depth" : [8, 16, 32],
  "min_samples_split" : [18, 16, 32],
  "min_samples_leaf" : [8, 16, 32],
  "criterion" : ["gini", "entropy"]
}

optimized = optimization(model_rf, param_grid, df, 5, 10)

optimized

{'criterion': 'gini',
 'max_depth': 32,
 'min_samples_leaf': 8,
 'min_samples_split': 32,
 'n_estimators': 64}

In [8]:
#Inserindo os valores otimizados no modelo
model_rf_optimized = RandomForestClassifier(criterion = optimized['criterion'],
                                            max_depth = optimized['max_depth'],
                                            min_samples_leaf = optimized['min_samples_leaf'],
                                            min_samples_split = optimized['min_samples_split'],
                                            n_estimators = optimized['n_estimators'])

run_model_cv(model_rf_optimized, df, 5, 10)

Mean train AUC: 0.934921444728686. Mean test AUC: 0.8053533288072764


(0.934921444728686, 0.8053533288072764)

#**Seção 3.2:**

#**Salvando o modelo**

#Para encerrar o trabalho, o modelo treinado pode ser salvo em um arquivo e ser encaminhado para utilização na produção. O modelo a ser utilizado será o *random forest*.

In [15]:
from joblib import dump, load
from google.colab import files

dump(model_rf_optimized, 'model_rf_optimized.joblib')
files.download('model_rf_optimized.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>